# Start Here

In [25]:
# static path variables
paths = {
    "ASSIST09" : "data/ASSISTments2009/",
}
wranglers = {
    "ASSIST09" : "data/ASSISTments2009/assistments09_wrangler", 
}
models = {
    "IRT" : "models/IRT/IRT",
}
prepper_path = "data/dataPrepper"

In [26]:
import os

def checkExists(path):
    #remove the .py file if it exists
    if(os.path.exists(path)):
        print("found " + path)
        return True
    else:
        print("couldn't find " + path)
        print(f"Try running: !jupyter nbconvert --to script {path[:-2] + 'ipynb'}")
        return False

In [43]:
# A temporary measure while working in notebooks
# Ensure all necesary scripts have been converted from notebooks
allExists = True

allExists = checkExists(prepper_path + ".py") and allExists

for key in models.keys():
    allExists = checkExists(models[key] + ".py") and allExists

for key in wranglers.keys():
    allExists = checkExists(wranglers[key] + ".py") and allExists

if(not allExists):
    print("\nMissing critical .py file(s), see above")

found data/dataPrepper.py
found models/IRT/IRT.py
found data/ASSISTments2009/assistments09_wrangler.py


In [42]:
!jupyter nbconvert --to script data/dataPrepper.ipynb

[NbConvertApp] Converting notebook data/dataPrepper.ipynb to script
[NbConvertApp] Writing 2927 bytes to data\dataPrepper.py


In [44]:
modelList ={
    "1":"IRT",
}

datasetList = {
    "1":"ASSIST09",
}

runTypeList = {
    "1":"basic"
}

model = modelList[input(f"Select a model to test:\n" + "\n".join([f'{k}.{modelList[k]}' for k in modelList.keys()]))]
dataset = datasetList[input(f"Select a dataset to test on:\n" + "\n".join([f'{k}.{datasetList[k]}' for k in datasetList.keys()]))]
runType = runTypeList[input(f"Select a run condition:\n" + "\n".join([f'{k}.{runTypeList[k]}' for k in runTypeList.keys()]))]

print(f"Performing test run of {model} using {dataset} in {runType} format")

Select a model to test:
1.IRT 1
Select a dataset to test on:
1.ASSIST09 1
Select a run condition:
1.basic 1


Performing test run of IRT using ASSIST09 in basic format


In [46]:
from data import dataPrepper as prep
from models.IRT import IRT

prep.prepareData(dataset = dataset, runType = runType, model = model)

Could not find Q matrix, calling wrangler
Wrangling Data


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Andrew\\Documents\\AJT WKU\\Thesis\\Graduate_Thesis\\Testing\\assistments_2009_2010.csv'